In [1]:
import sys
from pathlib import Path

root = Path.cwd().resolve().parent
if str(root) not in sys.path:
    sys.path.append(str(root))

print("Project root:", root)


Project root: C:\Users\ng4bo\OneDrive\Desktop\Alpaca Project\alpaca-bot-starter\alpaca-bot-starter


In [2]:
from src.config_strategy import TIER1_SYMBOLS
from src.modeling_arima import forecast_next_return
from pathlib import Path
import pandas as pd

DATA_DIR = Path("../data")

signals = []

for sym in TIER1_SYMBOLS:
    print(f"\n--- Processing {sym} ---")
    path = DATA_DIR / f"{sym}_1Day_returns_only.csv"
    df = pd.read_csv(path, parse_dates=["ts"])
    df = df.sort_values("ts").dropna(subset=["return"]).reset_index(drop=True)

    series = df["return"]
    pred = forecast_next_return(series, order=(1, 0, 1))  # same order you just used for SPY

    print("Forecast:", pred)

    signals.append({
        "symbol": sym,
        "forecast_return": pred,
        "n_points": len(series),
    })

signals_df = pd.DataFrame(signals)
signals_df



--- Processing DIA ---
Forecast: -0.0005571668036003447

--- Processing SPY ---
Forecast: -0.0017254139706869911

--- Processing XLF ---
Forecast: -7.434097472846829e-05

--- Processing QQQ ---
Forecast: -0.002578295131788406

--- Processing PG ---
Forecast: 0.0009576788791851846


,symbol,forecast_return,n_points
0,DIA,-0.000557,1975
1,SPY,-0.001725,1975
2,XLF,-0.000074,1975
3,QQQ,-0.002578,1975
4,PG,0.000958,1975


In [3]:
UP = 0.0008     # 0.08%
DOWN = -0.0008  # -0.08%

def classify_signal(r):
    if r > UP:
        return "long"
    elif r < DOWN:
        return "short"
    else:
        return "flat"

signals_df["signal"] = signals_df["forecast_return"].apply(classify_signal)
signals_df


,symbol,forecast_return,n_points,signal
0,DIA,-0.000557,1975,flat
1,SPY,-0.001725,1975,short
2,XLF,-0.000074,1975,flat
3,QQQ,-0.002578,1975,short
4,PG,0.000958,1975,long


In [4]:
import numpy as np

LONG_EXPOSURE = 0.20   # +20%
SHORT_EXPOSURE = -0.20 # -20%

signals_df["weight"] = 0.0

# Longs share LONG_EXPOSURE equally
mask_long = signals_df["signal"] == "long"
n_long = mask_long.sum()
if n_long > 0:
    signals_df.loc[mask_long, "weight"] = LONG_EXPOSURE / n_long

# Shorts share SHORT_EXPOSURE equally
mask_short = signals_df["signal"] == "short"
n_short = mask_short.sum()
if n_short > 0:
    signals_df.loc[mask_short, "weight"] = SHORT_EXPOSURE / n_short

signals_df


,symbol,forecast_return,n_points,signal,weight
0,DIA,-0.000557,1975,flat,0.0
1,SPY,-0.001725,1975,short,-0.1
2,XLF,-0.000074,1975,flat,0.0
3,QQQ,-0.002578,1975,short,-0.1
4,PG,0.000958,1975,long,0.2
